# Generating team and salaries data

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.ticker as ticker
from scipy.stats import linregress


In [ ]:
#Teams data import

teams_df = pd.read_csv("Baseball Dataset\Teams.csv")
                          
teams_df.head()

#Limiting data to years 1985 - 2015
teams_df = teams_df[teams_df['yearID']>=1985].reset_index(drop=True)

teams_df.head()

teams_df.groupby('franchID').count()

#Limiting to franchise in full sample
    #Note: The Arizona Diamondbacks, Colorado Rockies, Flordia Marlins, and Tampa Bay Devils franchises 
    #did not exist during the full study time perion (1985-2015) and are dropped from the dataset

excluded_franchID = ['ARI','COL','FLA','TBD']

teams_df = teams_df.loc[~teams_df['franchID'].isin(excluded_franchID)]

teams_df.groupby('franchID').count()


In [ ]:
#Salaries data import

salaries_df = pd.read_csv("Baseball Dataset\Salaries.csv")
                          
salaries_df.head()



In [ ]:
#Team player salaries summary statistics
team_salary_stats = salaries_df.groupby(['yearID','teamID']).agg(total_player_salaries = ('salary', 'sum'),
                                                                  med_player_salary = ('salary','median'),
                                                                  min_player_salary = ('salary','min'),
                                                                  max_player_salary = ('salary','max'),
                                                                  salary_var = ('salary','var'))

team_salary_stats.head()

In [ ]:
#Checking # of teams per year
team_salary_stats.groupby('yearID').nunique()

#Consider restricting our sample to original 26 teams


In [ ]:
#Merging Team and Salary Data
team_salary_df = pd.merge(teams_df, team_salary_stats, on = ('yearID','teamID'))

team_salary_df.head() 

# Export the merged data into a csv
team_salary_df.to_csv("Output/team_salary_merged.csv", index=False)
    

In [ ]:
#Adding CBT thresholds and flags for 2003-2015

thresholds = pd.read_csv("Salary Thresholds/2003-2015 Salary Thresholds.csv", thousands=',')

team_salary_2003_2015 = pd.merge(team_salary_df,thresholds, left_on='yearID', right_on='Year')

team_salary_2003_2015['over_threshold'] =np.where(team_salary_2003_2015['total_player_salaries']>team_salary_2003_2015['Threshold'], 1,0)

team_salary_2003_2015['times_over_threshold'] = team_salary_2003_2015.groupby('franchID')['over_threshold'].cumsum()

team_salary_2003_2015.head()

In [ ]:
#Dataset info
team_salary_2003_2015.info() 

team_salary_2003_2015.describe()

In [ ]:
rank_by_salary_df = teams_df.copy(deep=True)
rank_by_salary_df["Salary Rank"] = ""
rank_by_salary_df.head()

In [ ]:
# adding a salary difference column
diff_by_salary_df = team_salary_df.copy(deep=True)
diff_by_salary_df["Salary Rank"] = ""
diff_by_salary_df.head()

In [ ]:
team_salary_df['Salary Rank'] = team_salary_df.groupby("yearID")["total_player_salaries"].rank(ascending=False).astype(int)

team_salary_df.head()

In [ ]:
#Merging team, salary, and salary rank data

#yearly_team_salaries = yearly_team_salaries.drop(columns='total_player_salaries')

team_salary_df['Salary Rank'] = team_salary_df.groupby("yearID")["total_player_salaries"].rank(ascending=False).astype(int)

#Merging salary thresholds for 2003-2015

thresholds = pd.read_csv("Salary Thresholds/2003-2015 Salary Thresholds.csv", thousands=',')

team_salary_tax_df = pd.merge(team_salary_df,thresholds, left_on='yearID', right_on='Year', how='left').drop(columns=["Year"])

team_salary_tax_df['Threshold'].fillna(0,inplace=True)

#Creating Taxed (over_threshold) Flag for all years
top_5_years = [1997,1998,1999,2002]

team_salary_tax_df['over_threshold'] = np.where(np.logical_xor(np.logical_and(team_salary_tax_df['yearID'].isin(top_5_years), team_salary_tax_df['Salary Rank'] <=5),
                                                               np.logical_and(team_salary_tax_df['yearID'] >=2003, team_salary_tax_df['total_player_salaries']>team_salary_tax_df['Threshold'])),
                                                 1,0)
     #for 1997-2002 top 5 spending teams were taxed
     #for 2003-2015 teams were taxed if spent over threshold


team_salary_tax_df.head()



In [ ]:
#Counting # of times taxed since 1997
team_salary_tax_df['times_over_threshold'] = team_salary_tax_df.groupby('franchID')['over_threshold'].cumsum()

# Export the merged data into a csv
team_salary_tax_df.to_csv("Output/team_salary_tax.csv", index=False)

#Sorting by most to less taxed teams

df_2015 = team_salary_tax_df[team_salary_tax_df['yearID']==2015]
times_taxed = df_2015[['franchID', 'name', 'times_over_threshold']].sort_values('times_over_threshold', ascending=False)

times_taxed



In [ ]:
taxed_df = team_salary_tax_df[team_salary_tax_df['over_threshold']==1]

# Question 1 Analysis, salary impacts

In [ ]:
# group by year and franchID, and calculate the total player salaries for each group
yearly_team_salaries = team_salary_df.groupby(["yearID", "franchID"])["total_player_salaries"].sum().reset_index()

# Rank the total player salaries within each year
yearly_team_salaries['Salary Rank'] = yearly_team_salaries.groupby("yearID")["total_player_salaries"].rank(ascending=False).astype(int)

yearly_team_salaries.head(-30)

yearly_team_salaries[yearly_team_salaries["Salary Rank"]==1]

In [ ]:
# Rename the columns
yearly_team_salaries_rename = yearly_team_salaries.rename(columns={'yearID': 'Year', 'franchID': 'Franchise ID', 'total_player_salaries': 'Total Team Salary'})

yearly_team_salaries_rename

In [ ]:
# getting the difference between top and bottom each year
yearly_salary_difference = yearly_team_salaries_rename.groupby("Year")["Total Team Salary"].max() - yearly_team_salaries_rename.groupby("Year")["Total Team Salary"].min()

# getting the average salary per year

avg_salary_per_year = yearly_team_salaries_rename.groupby("Year")["Total Team Salary"].mean()

# bar chart
plt.figure(figsize=(10,6))
bar_colors=["blue" if year != 1997 else "red" for year in yearly_salary_difference.index]
yearly_salary_difference.plot(kind="bar", color=bar_colors)
plt.title("Disparity Between Teams with Highest and Lowest Salaries per Year")
plt.xlabel("Year")
plt.ylabel("Salary Difference, USD")
plt.xticks(rotation=45)
plt.tight_layout
formatter = ticker.StrMethodFormatter('{x:,.0f}')
plt.gca().yaxis.set_major_formatter(formatter)
plt.savefig("Output/Disparity Fig.png")
plt.show()

In [ ]:
avg_salary_per_year = yearly_team_salaries_rename.groupby("Year")["Total Team Salary"].mean().reset_index()

# Linear regression line
slope, intercept,_,_,_ = linregress(avg_salary_per_year["Year"], avg_salary_per_year["Total Team Salary"])
regression_line = slope * avg_salary_per_year["Year"] + intercept

# Plotting the regression chart
plt.figure(figsize=(10, 6))
plt.scatter(avg_salary_per_year["Year"], avg_salary_per_year["Total Team Salary"], color='blue', label="Average Total Team Salary")
plt.plot(avg_salary_per_year["Year"], regression_line, color="red", linestyle="-", label="Regression Line")
plt.title("Average Total Team Salary per Year (Regression)")
plt.xlabel("Year")
plt.ylabel("Average Total Team Salary, USD")
plt.grid(True, linestyle="--", alpha=0.7)
plt.legend()
plt.tight_layout()
plt.ticklabel_format(style='plain',axis="y")
formatter = ticker.StrMethodFormatter('{x:,.0f}')
plt.gca().yaxis.set_major_formatter(formatter)
plt.savefig("Output/Avg Regression Fig.png")
plt.show

# Question 2 Analysis, wins vs salary